# Task

Сегментировать договоры по тегам:
- <s1 - Условия договора
- <s2 - Запреты
- <s3 - Стоимость
- <s4 - Условия (относительно дат)
- <s5 - Штрафы
- <s6 - Адреса и местоположения


In [ ]:
!pip install -q pymorphy2           # установка лемматизатора pymorphy2

     |████████████████████████████████| 55 kB 2.1 MB/s 
     |████████████████████████████████| 8.2 MB 6.5 MB/s 


In [ ]:
!pip install tensorflow==2.3        # устанавливаем версию tf 2.3 и делаем restart runtime

In [ ]:
import numpy as np # Библиотека для работы с массивами данных
from tensorflow.keras.models import Model, load_model # Импортируем Model, load_model - метод, что загружает предобученную сеть
import re # Имортируем чтобы работать с строками
from tensorflow.keras.preprocessing.text import Tokenizer # Метод, который поволяет работать с текстами и конвертирует их в последовательности (индексов)
# Импорт слоёв нейронных сетей
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Conv1DTranspose, Lambda
from tensorflow.keras import backend as K # Импортируем, чтобы высчитать dice_coef(ошибку)
from tensorflow.keras.optimizers import Adam, Adadelta # Импортируем оптимизаторы
from tensorflow.keras import utils # Импортируем для работы с категориальными данными
import tensorflow
from google.colab import files # Импорт для работы с файлами
import matplotlib.pyplot as plt # Импорт для отрисовывания графиков
from gensim.models import word2vec # Импортируем gensim
import os # Импортируем для работы с системными файлами
import pandas as pd # Импортируем для работы с Массивами данных(таблицами - Датафреймами)
import time # Имортируем, чтобы высчитать время работы каких-либо процессов
import nltk #Natural language toolkit - Инструментарий естественного языка
from nltk.stem import WordNetLemmatizer  # Импортируем для работы с леммами
import pymorphy2 # Импортируем для работы с леммами

from tensorflow.keras.utils import  plot_model

nltk.download('wordnet') # Скачиваем сетку слов для лемматизации

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# mount the drive
from google.colab import drive
drive.mount('/content/my_drive')

Mounted at /content/my_drive


# Data Parsing

In [ ]:
########################
#clean the text
########################
def readText(fileName):
  f = open(fileName, 'r')
  text = f.read()
  delSymbols =  ['\n', "\t", "\ufeff", ".", "_", "-", ",", "!", "?", "–", "(", ")", "«", "»", "№", ";",'•','%',':']

  for dS in delSymbols:
    text = text.replace(dS, '')

  text = re.sub('<',' <',text)
  text = re.sub('>','> ',text)

  text = ' '.join(text.split())

  text = text.lower()
  return text


In [ ]:
########################
# lemmatizing the text. Sample: гулял -> гулять (нач. ф)
########################
def textToWords(text):
  morph = pymorphy2.MorphAnalyzer()
  words = text.split(' ')
  docs = [morph.parse(word)[0].normal_form for word in words]
  return docs


In [ ]:
directory = '/content/my_drive/MyDrive/Базы/Договора432/'

In [ ]:
agreements = []
for filename in os.listdir(directory):
  txt = readText(directory + filename)
  if txt != '':
    agreements.append(txt)

In [ ]:
n = 3
print(os.listdir(directory)[n]) # Выводим названия n-ого договора
agreements[n]                   # Текст n-ого договора

2 (5).txt


'договор пожизненного содержания с иждивением с предоставлением иного жилого помещения для проживания взамен передаваемого г г гражданин паспорт серия номер выдан проживающий по адресу именуемый в дальнейшем получатель ренты с одной стороны и гражданин паспорт серия номер выдан проживающий по адресу именуемый в дальнейшем плательщик ренты с другой стороны именуемые в дальнейшем стороны заключили настоящий договор в дальнейшем договор о нижеследующем 1 предмет договора11 <s1> получатель ренты передает плательщику ренты в собственность квартиру <s6> расположенную по адресу </s6> именуемую в дальнейшем передаваемое помещение а плательщик ренты в обмен на полученное помещение предоставляет получателю ренты квартиру комнату для проживания и <s3> обязуется осуществлять пожизненное содержание с иждивением получателя ренты под содержанием с иждивением в настоящем договоре понимается обеспечение потребностей получателя ренты в жилище питании и одежде уход за ним оплата ритуальных услуг </s1> </

In [ ]:
time1 = time.time()
docs_full = []
for i in agreements:
  docs_full.append(textToWords(i))
print(time.time() - time1)

200.51403641700745


In [ ]:
print('number of all texts :',len(docs_full))
docs = docs_full[0:-10]
docsToTest = docs_full[-10:]
print('number of train texts :',len(docs))
print('number of train test :',len(docsToTest))
print('Sample :', docs[0][0:10])

number of all texts : 422
number of train texts : 412
number of train test : 10
Sample : ['договор', 'безвозмездный', 'хранение', 'до', 'востребование', 'г', 'г', 'гражданин', 'паспорт', 'серия']


# Preprocessing

In [ ]:
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)

tokenizer.fit_on_texts(docs_full) 
clean_voc = {}                    

for item in tokenizer.word_index.items(): # 
  clean_voc[item[0]] = item[1]            # [('unknown', 1), ('start', 2), ('end', 3)] -> {'unknown': 1, 'start': 2, 'end': 3}

In [ ]:
print('Frequence dictionary :') 
print(clean_voc, '\n')
print('Length of dict:', len(clean_voc))

tag = '<s1>'
print('Ibdex of', tag, ':' ,clean_voc[tag])


Frequence dictionary :
{'в': 1, 'и': 2, 'договор': 3, 'по': 4, 'с': 5, 'настоящий': 6, 'на': 7, 'сторона': 8, '<s1>': 9, '</s1>': 10, 'не': 11, 'или': 12, 'за': 13, 'он': 14, 'случай': 15, 'о': 16, '<s3>': 17, '</s3>': 18, 'право': 19, 'работа': 20, 'срок': 21, '<s4>': 22, '</s4>': 23, 'от': 24, 'они': 25, 'при': 26, 'другой': 27, 'быть': 28, 'действовать': 29, 'день': 30, 'арендатор': 31, 'имущество': 32, 'для': 33, 'лицо': 34, 'к': 35, 'если': 36, 'дальнейший': 37, 'условие': 38, 'который': 39, 'обязательство': 40, 'страховой': 41, 'а': 42, 'работник': 43, 'сумма': 44, 'размер': 45, 'порядок': 46, 'действие': 47, 'свой': 48, 'соответствие': 49, 'арендодатель': 50, 'указанный': 51, 'законодательство': 52, 'счёт': 53, 'мочь': 54, 'именовать': 55, 'из': 56, 'основание': 57, 'один': 58, 'исполнение': 59, 'страховщик': 60, 'обязанность': 61, 'иной': 62, 'страхователь': 63, 'до': 64, 'также': 65, 'иметь': 66, 'трудовой': 67, 'обязать': 68, 'год': 69, 'это': 70, 'вправе': 71, 'ответственнос

In [ ]:
# Превращает текст в последовательность индексов согласно словарю частотности
tok_agreem = tokenizer.texts_to_sequences(docs) # Обучающий выборка в последовтельность индексов

print("Взгляните на фрагмент обучающей выборки:")
print("Исходный текст:              ", docs[4][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreem[4][:20], '\n')

Взгляните на фрагмент обучающей выборки:
Исходный текст:               ['договор', 'бартер', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'сторона', '1', 'с', 'один', 'сторона', 'и', 'в', 'лицо']
Тот же текст, но как последовательность индексов:  [3, 5813, 81, 81, 1, 34, 29, 7, 57, 55, 1, 37, 8, 110, 5, 58, 8, 2, 1, 34] 



## -> yTrain

In [ ]:
def getXYSamples(tok_agreem,tags_index):    # tok_agreem - [[3, 5813, 81 ...],[...],[...]], tags_index - 12 indexes of tags
  tags01 =[]
  indexes = []

  for agreement in tok_agreem:
    tag_place = [0,0,0,0,0,0]
    for ex in agreement:
      if ex in tags_index:
        place = np.argwhere(tags_index==ex)       # index of tag in tags_index list 
        if len(place) != 0:
          if place[0][0] < 6:
            tag_place[place[0][0]]=1              # [0,0,0,0,0,0] -> [0,0,1,0,0,0]
          else:
            tag_place[place[0][0] - 6]=0          # [0,0,1,0,0,0] -> [0,0,0,0,0,0]
      else:
        tags01.append(tag_place.copy())           # [[0,0,1,0,0,0],[0,0,1,0,0,0]...[0,0,0,0,0,0]] - yTrain
        indexes.append(ex)                        # [45,68,985,3006,10...] - xTrain

  return (indexes,tags01)



In [ ]:
########################
#  [45,68,985,3006,10...] -> ['договор', 'купить', 'нельзя'.......]
########################
def reverseIndex(clean_voc, x):
  reverse_word_map = dict(map(reversed, clean_voc.items())) #  Берёт ключ и значение из словаря и меняет их местами
  docs = [reverse_word_map.get(letter) for letter in x] # Положим их в лист
  return docs # Возвращает текст

In [ ]:
########################
# list of tags
########################

#<s1> - Условия договора
#<s2> - Запреты
#<s3> - Стоимость
#<s4> - Условия (относительно дат)
#<s5> - Штрафы
#<s6> - Адреса и местоположения

tags_index = ['<s' + str(i) + '>' for i in range(1, 7)]   # Лист открывающих тегов
closetags = ['</s' + str(i) + '>' for i in range(1, 7)]   # Лист закрывающих тегов
tags_index.extend(closetags)                              # конкатенируем все теги

tags_index = np.array([clean_voc[i] for i in tags_index]) # Получаем инексы всех тегов из словря частотности
print('Индексы всех тегов:', tags_index)

Индексы всех тегов: [  9 330  17  22  99 254  10 338  18  23 100 248]


In [ ]:
########################
# xData / yData / embedding list
########################
curTime = time.time()
xData, yData = getXYSamples(tok_agreem,tags_index)
decoded_text = reverseIndex(clean_voc, xData) # [45,68,985,3006,10...] -> ['договор', 'купить', 'нельзя'.......]
print('Превращение заняло: ', round(time.time() - curTime, 2), 'с.')

Превращение заняло:  3.61 с.


In [ ]:
########################
# check point
########################
print('Lenght of xData :',len(xData))
print('Lenght of yData :',len(yData))

print('Source words: ', docs[0][50:90])
print('Decoded words: ', decoded_text[0][50:90])

print('Part of xData',xData[50:90])
print('Part of yData',yData[50:90])

Lenght of xData : 380336
Lenght of yData : 380336
Source words:  ['1', 'предмет', 'договора11', '<s1>', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '</s1>', '12', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор', 'следующий', 'вещь', 'далее', 'вещь13', '<s1>', 'при', 'передача', 'вещь', 'поклажедатель', 'на', 'хранение']
Decoded words:  
Part of xData [110, 97, 292, 4, 6, 3, 206, 122, 932, 318, 112, 14, 238, 2, 548, 220, 318, 1, 399, 251, 238, 293, 7, 149, 4, 6, 3, 138, 318, 310, 5769, 26, 162, 318, 238, 7, 149, 206, 8, 843]
Part of yData [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [1

## Windowing method

In [ ]:
########################
# [[xLen] -step- [xLen],[xLen],[xLen]...]
########################
def getSetFromIndices(wordIndices, xLen, step): 
  xBatch = [] 
  wordsLen = len(wordIndices)
  index = 0 
  
  while (index + xLen <= wordsLen): 
    xBatch.append(wordIndices[index:index+xLen]) # [[110, 97, 292...] -> [[xLen] -step- [xLen],[xLen],[xLen]...]
    index += step # next step

  return xBatch 

In [ ]:
########################
# xTrain / yTrain
########################
xLen = 256 
step = 30 
embeddingSize = 300 # The number of dimensions of the vector space for each of our words

# create xTrain and yTrain by Windowing method
xTrain = getSetFromIndices(decoded_text, xLen, step) 
yTrain = getSetFromIndices(yData, xLen, step) 

print('Lenght xTrain:', len(xTrain))
print('Lenght yTrain:', len(yTrain))
print('Lenght Sample from xTrain:',len(xTrain[0]))
print('Lenght Sample from yTrain:',len(yTrain[0]), '\n')
print('Sample xTrain', xTrain[0])
print('Sample yTrain', yTrain[0], '\n')

print('First Sample xTrain:', xTrain[0][step-5:step+5])
print('Second Sample xTrain:', xTrain[1][:10])

Lenght xTrain: 12670
Lenght yTrain: 12670
Lenght Sample from xTrain: 256
Lenght Sample from yTrain: 256 

Sample xTrain ['договор', 'безвозмездный', 'хранение', 'до', 'востребование', 'г', 'г', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'хранитель', 'с', 'один', 'сторона', 'и', 'гражданин', 'паспорт', 'серия', 'номер', 'выдать', 'проживать', 'по', 'адрес', 'именовать', 'в', 'дальнейший', 'поклажедатель', 'с', 'другой', 'сторона', 'именовать', 'в', 'дальнейший', 'сторона', 'заключить', 'настоящий', 'договор', 'в', 'дальнейший', 'договор', 'о', 'нижеследующий', '1', 'предмет', 'договора11', 'по', 'настоящий', 'договор', 'хранитель', 'обязываться', 'хранить', 'вещь', 'передать', 'он', 'поклажедатель', 'и', 'возвратить', 'этот', 'вещь', 'в', 'сохранность', '12', 'поклажедатель', 'передавать', 'на', 'хранение', 'по', 'настоящий', 'договор', 'следующий', 'вещь', 'далее', 'вещь13', 'при', 'передача', 'вещь', 'поклажедатель',

## Tokenization

In [ ]:
# Превращаем текст в последовательность индексов согласно словарю частотности
tok_agreemTest = tokenizer.texts_to_sequences(docsToTest) # Обучающий текст в индексы

print("Посмотрим на фрагмент тестового текста:")
print("Исходный текст:              ", docsToTest[4][:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreemTest[4][:20], '\n')

xDataTest, yDataTest = getXYSamples(tok_agreemTest,tags_index) # Распознаём теги и создаём лист, что их хранят
decoded_text = reverseIndex(clean_voc, xDataTest) # Чтобы создать лист для эмбеддинга, нам нужно превратить лист индексов обратно в лист слов
print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))

print('Сдекодированные текст:', decoded_text[50:80])
print('Часть xDataTest:     ', xDataTest[50:80])
print('Часть yDataTest:     ', yDataTest[50:80])

xLen = 256 # Длина окна
step = 30 # Шаг 
embeddingSize = 300 # Число измерений вектор-пространства для каждого нашего слова 

# Генерируем выборки с определёнными параметрами
xTest = getSetFromIndices(decoded_text, xLen, step) # Последовательность индексов с окном Xlen слов
yTest = getSetFromIndices(yDataTest, xLen, step) # # Последовательность индексов с окном Xlen от тегов

print('Длина xTest:', len(xTest))
print('Длина yTest:', len(yTest))
print('Длина примера xTest:',len(xTest[0]))
print('Длина примера yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')

print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Посмотрим на фрагмент тестового текста:
Исходный текст:               ['трудовой', 'контракт', 'с', 'специалист', 'г', 'г', 'в', 'лицо', 'действовать', 'на', 'основание', 'именовать', 'в', 'дальнейший', 'работодатель', 'с', 'один', 'сторона', 'и', 'гражданин']
Тот же текст, но как последовательность индексов:  [67, 209, 5, 1001, 81, 81, 1, 34, 29, 7, 57, 55, 1, 37, 127, 5, 58, 8, 2, 275] 

Длина xDataTest: 11823
Длина yDataTest: 11823
Сдекодированные текст: ['предмет', 'договора11', 'работник', 'приниматься', 'на', 'работа', 'на', 'предприятие', 'на', 'должность', 'руководитель', 'служба', 'реклама', 'и', 'маркетинг', 'для', 'выполнение', 'трудовой', 'обязанность', 'связанный', 'с', 'руководство', 'указанный', 'служба', 'планирование', 'и', 'стимулирование', 'сбыт', 'изучение', 'и']
Часть xDataTest:      [97, 292, 43, 1023, 7, 20, 7, 258, 7, 522, 702, 660, 1210, 2, 2012, 33, 83, 67, 61, 266, 5, 908, 51, 660, 2546, 2, 4333, 2705, 2988, 2]
Часть yDataTest:      [[0, 0, 0, 0, 0, 0], [0, 0

## Loss Function

In [ ]:
########################
# for the accuracy metric
########################
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)
    

## Create xTrain / yTrain by Word2Vec GENSIM

In [ ]:
def pad_zeros(phrase, xLen = 256): # Функция для дополнением нулями
  while len(phrase) < xLen: # Пока сущестующий размер меньше чем требуемый 
    phrase.append([0] * embeddingSize) # Продолжаем добавлять элементы(нули) в фразу(zero-pad)  
  if len(phrase) > xLen: # В противном случае, если фраза больше чем  требуемый размер
    phrase = phrase[:xLen] # Берём только требуемое число элементов
  return phrase # Возвращаем обработанную фразу

# Создаём выборку
def getSets(model, senI, tagI):
  xVector = [] # Лист будет содержать ембеддинг представление на каждый индекс
  tmp = [] # Временный список
  for text in senI: # Проходимся по каждому лист-тексту
    tmp=[]
    for word in text: # Проходимся по кажому слову лист-текста
      try: # Если слово есть в модели (словарь не слишком мал), мы не получим ошибку
        tmp.append(model[word])
      except: # В противном случае пропустим слово
        pass

    xVector.append(pad_zeros(tmp, xLen))
  temp = np.array(xVector)
  return np.array(xVector, dtype = np.float32), np.array(tagI)

In [ ]:
########################
# create modelGENSIM be Word2Vec (pre-trained Embedding)
########################

# Подаём лист листов слов в word2vec для обучающей выборке
# size = embeddingSize - Размер ембеддинга
# window = 10 -  минимальное расстояния между эмбеддинг словами
# min_count = 1 - Игнорируем все слова с частотой меньше чем 1
# workers = 10 -  Число потоков на обучение эмбеддинга
# iter = 20 -  Число эпох на обучение эмбеддинга
# max_vocab_size = 1e5 -  Число слов в "словарном запасе" word2vec
curTime = time.time()

##########################################################################################################
# modelGENSIM  contains all our words in the form of codes of a multidimensional space
modelGENSIM = word2vec.Word2Vec(xTrain + xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10, max_vocab_size = 1e5)
print('Обучение GENSIM модели заняло: ', round(time.time() - curTime, 2), 'с.')

Обучение GENSIM модели заняло:  77.62 с.


In [ ]:
GENSIMtrainX, GENSIMtrainY = getSets(modelGENSIM, xTrain, yTrain)
GENSIMtestX, GENSIMtestY = getSets(modelGENSIM, xTest, yTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [ ]:
print('Размерности xTrain:', GENSIMtrainX.shape)
print('Размерности yTrain:', GENSIMtrainY.shape)

Размерности xTrain: (12670, 256, 300)
Размерности yTrain: (12670, 256, 6)


In [ ]:
modelGENSIM['зарплата'] # Пример превращения слова в вектор при помощи GENSIM

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.12148635,  0.04060454, -0.05649525,  0.04235208,  0.3406356 ,
       -0.30001003,  0.18589652, -0.06688875,  0.22348872, -0.1544004 ,
        0.20178641,  0.16322315, -0.2960511 , -0.4525736 , -0.31769297,
        0.10781555,  0.06059976, -0.09641528,  0.00981096,  0.02318526,
        0.03028118, -0.19761577,  0.09493009, -0.37196574,  0.0077421 ,
       -0.3379384 , -0.35514805, -0.17200501, -0.06179683,  0.00414547,
        0.19190209, -0.13849586,  0.08069058, -0.10509472, -0.33408374,
        0.11269737, -0.05231242, -0.19116943, -0.31297815, -0.13379695,
        0.0516334 ,  0.07179373,  0.16810237,  0.01695984, -0.14694451,
       -0.04803488, -0.33197153, -0.33820003, -0.27821448,  0.04637684,
       -0.05323692,  0.04222822,  0.12406273,  0.01444374, -0.09078339,
        0.03865733,  0.00518312,  0.23119184, -0.07182365,  0.2882926 ,
       -0.22877944, -0.01179283, -0.19489585, -0.02527171,  0.04801432,
        0.10173117, -0.24894659,  0.17372818, -0.12336968, -0.06

# Modelling

In [ ]:
# Функция по созданию сети UNET
# kernel_size = 3
# + step
def create_unet(k = 1, kernel_size = 3, num_classes = 6, input_shape= (30, 300)):
    img_input = Input(input_shape) 

    # Блок 1
    x = Conv1D(64 * k , kernel_size, padding='same')(img_input) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k, kernel_size, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    # Блок 2
    x = MaxPooling1D()(block_1_out)
    x = Conv1D(128 * k, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    # Блок 3
    x = MaxPooling1D()(block_2_out)
    x = Conv1D(256 * k, kernel_size, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # Блок 4
    x = MaxPooling1D()(block_3_out)
    x = Conv1D(512 * k, kernel_size, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out

     # Блок 5
    x = MaxPooling1D()(block_4_out)
    x = Conv1D(768 * k, kernel_size, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(768 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(768 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)      
    block_5_out = Activation('relu')(x)
    x = block_5_out   

########################
############
# 
############
########################

    # UP 1
    x = Conv1DTranspose(512 * k, kernel_size=kernel_size, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_4_out]) 
    x = Conv1D(512 * k , kernel_size, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 2
    x = Conv1DTranspose(256 * k, kernel_size=kernel_size, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , kernel_size, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128 * k, kernel_size=kernel_size, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64 * k, kernel_size=kernel_size, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , kernel_size, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, kernel_size, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

In [ ]:
model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
history = model_b_UNET.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=15, batch_size=64) # Feed the images divided among 2 classes to the model

Epoch 1/15
198/198 [==============================] - 40s 203ms/step - loss: 0.5779 - dice_coef: 0.5469 - val_loss: 3.0934 - val_dice_coef: 0.2520
Epoch 2/15
198/198 [==============================] - 39s 195ms/step - loss: 0.3619 - dice_coef: 0.6570 - val_loss: 1.5225 - val_dice_coef: 0.3987
Epoch 3/15
198/198 [==============================] - 39s 195ms/step - loss: 0.3022 - dice_coef: 0.6939 - val_loss: 1.2115 - val_dice_coef: 0.4702
Epoch 4/15
198/198 [==============================] - 39s 195ms/step - loss: 0.2728 - dice_coef: 0.7135 - val_loss: 1.4853 - val_dice_coef: 0.4633
Epoch 5/15
198/198 [==============================] - 39s 195ms/step - loss: 0.2590 - dice_coef: 0.7272 - val_loss: 0.7865 - val_dice_coef: 0.5390
Epoch 6/15
198/198 [==============================] - 39s 195ms/step - loss: 0.2490 - dice_coef: 0.7376 - val_loss: 1.0384 - val_dice_coef: 0.4866
Epoch 7/15
198/198 [==============================] - 39s 195ms/step - loss: 0.2449 - dice_coef: 0.7433 - val_loss: 1.

In [ ]:
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_UNET, tags, xLen, 0.1) # Проверяем UNET архитектуру

Сеть распознала категорию  'S1' с точностью в 77.43%
Сеть распознала категорию  'S2' с точностью в 97.24%
Сеть распознала категорию  'S3' с точностью в 82.76%
Сеть распознала категорию  'S4' с точностью в 83.12%
Сеть распознала категорию  'S5' с точностью в 92.55%
Сеть распознала категорию  'S6' с точностью в 99.41%
Средняя точность 88.75%
